MIT License

Copyright (c) 2017 Ryan Whytsell

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.


Modified by Harish Iyer Ganesan

In [1]:
import sys
import pygame
import random
from threading import Thread
#import PreyPredator

import warnings
warnings.filterwarnings("ignore")

pygame 2.1.0 (SDL 2.0.16, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import random
import sys
import copy
random.seed(1)

In [3]:
import csv


In [4]:
class Node:

    species = None
    x = None
    y = None
    health = None
    

    def __init__(self, spec=None):
        if spec is not None:
            self.species = spec
        else:
            self.species = 0
        self.x = None
        self.y = None
        #self.health = self.species*2
        if spec is 0:  # Blank
            self.health = None
        if spec is 1:  # Predator
            self.health = 3
        if spec is 2:  #Intermediate-Predator
            self.health = 3
        if spec is 3:  # Prey
            self.health = 4

    def set_location(self, pos_x, pos_y):
        self.x = pos_x
        self.y = pos_y

    def set_species(self, num):
        if num == 0 or num == 1 or num == 2 or num == 3:
            self.species = num
            if num is 0:  # Blank
                self.health = None
            if num is 1:  # Predator
                self.health = 2
            if num is 2:  #Intermediate-Predator
                self.health = 2
            if num is 3:  # Prey
                self.health = 4
        else:
            sys.stderr.write('Species not correct, do not do math with them.')
            self.species = 0

    def move_here(self, species, health):
        if species == 0 or species == 1 or species == 2 or species == 3:
            self.species = species
            if species is 0:  # Blank
                self.health = health
            if species is 1:  # Predator
                self.health = health
            if species is 2:  #Intermediate-Predator
                self.health = health
            if species is 3:  # Prey
                self.health = health
        else:
            sys.stderr.write('Species not correct, do not do math with them.')
            self.species = 0

    def prey_eat(self):
        self.species = 1
        #self.health = 2
            
    def predator_eat_intermediate_predator(self):
        self.species = 1
        #self.health = 2
            
    def intermediate_predator_prey_eat(self):
        self.species = 2
        #self.health = 3

            
    def intermediate_predator_reproduce(self):
        self.species = 2
        #self.health = 3

    def prey_reproduce(self):
        self.species = 3
        self.health = 4

In [5]:
class Map:
    height = 100
    width = 100
    play_board = []
    kill_count = 0
    prey_kill_count = 0
    
    def __init__(self):
        
        prey = 0
        predator = 0
        intermediate_predator = 0
        empty = 0       
        for x in range(self.width):
            row = []
            
            # Random Spatial Dostribution
            for y in range(self.height):
                i = random.randint(0, 100)
                if i <= 40:
                    row.append(Node(3))
                    prey += 1
                elif i <= 60 :
                    row.append(Node(2))
                    intermediate_predator += 1
                elif i<= 80:
                    row.append(Node(1))
                    predator += 1
                else:
                    row.append(Node(0))
                    empty += 1
               
#             #Clustered Spatial Dostribution
#             for y in range(self.height):
#                 i = random.randint(0, 100)
#                 if i <= 25 and x<50 and y<50:
#                     row.append(Node(3))
#                     prey += 1
#                 elif i <= 25 and x>50 and y>50:
#                     row.append(Node(3))
#                     prey += 1
#                 elif i <= 50 and x<50 and y<50:
#                     row.append(Node(2))
#                     intermediate_predator += 1
#                 elif i <= 75 and x>50 and y>50:
#                     row.append(Node(1))
#                     predator += 1
#                 else:
#                     row.append(Node(0))
#                     empty += 1

            self.play_board.append(row)
        print('Map created')
        print('Prey: ', prey)
        print('Intermediate-Predator: ', intermediate_predator)
        print('Predator: ', predator)
        print('Empty: ', empty)
        print('Width: ', len(self.play_board))
        print('Height of row 5: ', len(self.play_board[5]))
        #writer.writerow({'time_step': 0, 'prey': prey, 'intermediate_predator': intermediate_predator, 'predator': predator, 'empty': empty  })
    
    def map_count(self, time_step,writer):
        pb = self.play_board
        prey = 0
        predator = 0
        intermediate_predator = 0
        empty = 0
        for x in range(0, self.width):
            for y in range(0, self.height):
                if  pb[x][y].species is 3:
                    prey += 1
                elif pb[x][y].species is 2:
                    intermediate_predator += 1
                elif pb[x][y].species is 1:
                    predator += 1
                else:
                    empty += 1
                
                
        print('Time Step: ',time_step)
        print('Prey: ', prey)
        print('Intermediate-Predator: ', intermediate_predator)
        print('Predator: ', predator)
        print('Empty: ', empty)
        print('-----------------------------------------')
        writer.writerow({'time_step': time_step, 'prey': prey, 'intermediate_predator': intermediate_predator, 'predator': predator, 'empty': empty  })

    
    def get_board(self):
        return self.play_board

    def turn(self):
        pb = self.play_board
        [[self.check_neighbors(x, y, pb[x][y]) for y in range(0, self.height)] for x in range(0, self.width)]

    # Returns the neighbors of a node, If node is on edge this wraps
    def get_neighbors(self, x, y):
        top = self.play_board[x][(y - 1) % self.height]
        tr = self.play_board[(x + 1) % self.width][(y - 1) % self.height]
        right = self.play_board[(x + 1) % self.width][y]
        br = self.play_board[(x + 1) % self.width][(y + 1) % self.height]
        bottom = self.play_board[x][(y + 1) % self.height]
        bl = self.play_board[(x - 1) % self.width][(y + 1) % self.height]
        left = self.play_board[(x - 1) % self.width][y]
        tl = self.play_board[(x - 1) % self.width][(y - 1) % self.height]

        neighbors = [top, tr, right, br, bottom, bl, left, tl]
        #neighbors = [top, right, bottom, left]
        return neighbors

    #
    # PREDATOR AND PREY CELLULAR AUTOMATON
    # The world is grid of cells, with 4 possibilities: Predator(Red, 1), Intermediate-Predator(Yellow, 2) Prey(Blue, 3), or Empty(Black, 0).
    # All predator, intermediate-predator and prey have a set health, that changes over time.
    # The simulation works in steps, with the following rules:
    #    -For prey:
    #        -Tries to move in a random direction.
    #        -Health increases.
    #        -When health reaches a threshold:
    #           -They will reproduce, creating a new "Prey"
    #            -Their health resets to 1
    #    -For intermediate-predator:
    #        -Tries to move in a random direction.
    #        -Health decreases.
    #        -When health reaches 0, they die and turn into "Nothing".
    #       -If the adjacent square is a prey:
    #            -They will eat it, turning it into a "predator" (reproducing)
    #            -Their health will increase by the amount of health the eaten prey had
    #    -For predator:
    #        -Tries to move in a random direction.
    #        -Health decreases.
    #        -When health reaches 0, they die and turn into "Nothing".
    #       -If the adjacent square is a prey:
    #            -They will eat it, turning it into a "predator" (reproducing)     
    #            -Their health will increase by the amount of health the eaten prey had
    
    def check_neighbors(self, x, y, node):
        # If the node is empty no checks are performed
        if node.species is 0:
            return

        if node.species is 1:
            # This is a predator. Check for prey if no prey reduce health/die
            neighbors = self.get_neighbors(x, y)
            prey = []
            intermediate_predator = []
            open_ = []

            for neigh in neighbors:
                if neigh.species is 3:
                    prey.append(neigh)
                if neigh.species is 2:
                    intermediate_predator.append(neigh)
                if neigh.species is 0:
                    open_.append(neigh)
            open_length = len(open_)

            
            if len(prey) is 0:
                if node.health > 0 and open_length > 0:
                    node.health -= 1
                    open_[random.randint(0, (open_length-1))].move_here(1, node.health)
                    node.set_species(0)
                else:
                    node.health -= 1
                    
            elif len(prey) > 0:
                target = prey[random.randint(0, (len(prey)-1))]
                if node.health >= target.health:
                    target.prey_eat() 
                    self.prey_kill_count+=1

                       
                         
            if node.health is None or node.health <= 0:
                node.set_species(0)
                return

        if node.species is 2:
            # This is an intermediate-predator. Check for prey if no prey reduce health/die
            neighbors = self.get_neighbors(x, y)
            prey = []
            predator = []
            open_ = []

            for neigh in neighbors:
                if neigh.species is 3:
                    prey.append(neigh)
                if neigh.species is 1:
                    predator.append(neigh)
                if neigh.species is 0:
                    open_.append(neigh)
            open_length = len(open_)
            

            if len(prey) is 0:
                if node.health >0  and open_length > 0:
                    node.health -= 1
                    open_[random.randint(0, (open_length-1))].move_here(2, node.health)
                    node.set_species(0)
                else:
                    node.health -= 1
                    
            else:
                target = prey[random.randint(0, (len(prey)-1))]
                if node.health >= target.health:
                    target.intermediate_predator_prey_eat()
                    self.prey_kill_count+=1



            if node.health is None or node.health <= 0:
                node.set_species(0)
                return
            
#             if node.health > 4 and open_length >= 1:
#                 open_[random.randint(0, (open_length-1))].intermediate_predator_reproduce()
#                 node.health = 2
#                 return            
            
        if node.species is 3:
            # Check for overpopulation/health This is prey
            neighbors = self.get_neighbors(x, y)
            open_spots = []
            for neigh in neighbors:
                if neigh.species is 0:
                    open_spots.append(neigh)
            count = len(open_spots)
            if count >= 7 or count < 1:
                node.health -= 2
            if node.health is None or node.health is 0:
                node.set_species(0)
                return
            if node.health > 6 and len(open_spots) >= 1:
                open_spots[random.randint(0, (len(open_spots))-1)].prey_reproduce()
                node.health = 4
                return
            node.health += 1

In [6]:
#random.seed(1)
size = width, height = 500, 500
play_width = width/5
play_height = height/5
is_stopped = False

In [7]:
# These are the codes that pygame uses, I am not sure what format it is
black = 0
red = ((255,0,0))
yellow = ((255,255,0))
blue = ((0,0,255))

color_arr = [black, red, yellow, blue]

In [8]:
pygame.init()
screen = pygame.display.set_mode(size)
pp = Map()

Map created
Prey:  4060
Intermediate-Predator:  1986
Predator:  2011
Empty:  1943
Width:  100
Height of row 5:  100


In [9]:
# Paints a 3x3 square for each node
def paint(x, y, color_int):
    screen.set_at((x, y), color_arr[color_int])
    screen.set_at((x + 1, y), color_arr[color_int])
    screen.set_at((x, y + 1), color_arr[color_int])
    screen.set_at((x + 1, y + 1), color_arr[color_int])
    screen.set_at((x + 2, y), color_arr[color_int])
    screen.set_at((x, y + 2), color_arr[color_int])
    screen.set_at((x + 2, y + 2), color_arr[color_int])
    screen.set_at((x + 2, y + 1), color_arr[color_int])
    screen.set_at((x + 1, y + 2), color_arr[color_int])

In [10]:
# Creates random colors TESTING ONLY
def random_colors(w, h):
    ra = random.randint
    pa = paint
    while not is_stopped:
        for x in range(5, w, 5):
            for y in range(5, h, 5):
                color_int = ra(1, 2)
                pa(x, y, color_int)

In [11]:
# Paints the species at each point on the play board
def paint_map(board):
    pa = paint
    length = len(board)
    for x in range(0, length):
        for y in range(0, length):
            pa(x*5, y*5, board[x][y].species)

In [12]:
#### Game loop, paints the map then calculates the next turn
time_step=0
#open the file in the write mode
with open('1prey_2predator_NoIGP.csv', 'w', newline='') as csvfile:
    fieldnames = ['time_step', 'prey','intermediate_predator','predator','empty']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                is_stopped = True
                sys.exit(0)
        paint_map(pp.get_board())
        pygame.display.flip()
        pp.map_count(time_step, writer)
        pp.turn()
#         if(time_step==0 or time_step==200 or time_step==500 or time_step==1000):
#             a=input("Stop for snapshot")
        if(time_step==1000):
            print("Kill Count: ", pp.kill_count)
            print("Prey Kill Count: ",pp.prey_kill_count)
            pygame.QUIT
            a=input("Stop for snapshot")
        time_step+=1
            

Time Step:  0
Prey:  4060
Intermediate-Predator:  1986
Predator:  2011
Empty:  1943
-----------------------------------------
Time Step:  1
Prey:  3766
Intermediate-Predator:  2125
Predator:  2166
Empty:  1943
-----------------------------------------
Time Step:  2
Prey:  3447
Intermediate-Predator:  2301
Predator:  2307
Empty:  1945
-----------------------------------------
Time Step:  3
Prey:  3353
Intermediate-Predator:  2324
Predator:  2330
Empty:  1993
-----------------------------------------
Time Step:  4
Prey:  4775
Intermediate-Predator:  2440
Predator:  2465
Empty:  320
-----------------------------------------
Time Step:  5
Prey:  3807
Intermediate-Predator:  2850
Predator:  2935
Empty:  408
-----------------------------------------
Time Step:  6
Prey:  2439
Intermediate-Predator:  3439
Predator:  3538
Empty:  584
-----------------------------------------
Time Step:  7
Prey:  1095
Intermediate-Predator:  4078
Predator:  4241
Empty:  586
--------------------------------------

Time Step:  69
Prey:  1552
Intermediate-Predator:  364
Predator:  682
Empty:  7402
-----------------------------------------
Time Step:  70
Prey:  1596
Intermediate-Predator:  368
Predator:  706
Empty:  7330
-----------------------------------------
Time Step:  71
Prey:  1613
Intermediate-Predator:  364
Predator:  709
Empty:  7314
-----------------------------------------
Time Step:  72
Prey:  1605
Intermediate-Predator:  366
Predator:  725
Empty:  7304
-----------------------------------------
Time Step:  73
Prey:  1654
Intermediate-Predator:  354
Predator:  719
Empty:  7273
-----------------------------------------
Time Step:  74
Prey:  1727
Intermediate-Predator:  349
Predator:  744
Empty:  7180
-----------------------------------------
Time Step:  75
Prey:  1688
Intermediate-Predator:  345
Predator:  797
Empty:  7170
-----------------------------------------
Time Step:  76
Prey:  1717
Intermediate-Predator:  350
Predator:  804
Empty:  7129
-----------------------------------------


Time Step:  134
Prey:  2661
Intermediate-Predator:  523
Predator:  1632
Empty:  5184
-----------------------------------------
Time Step:  135
Prey:  2675
Intermediate-Predator:  539
Predator:  1613
Empty:  5173
-----------------------------------------
Time Step:  136
Prey:  2683
Intermediate-Predator:  539
Predator:  1628
Empty:  5150
-----------------------------------------
Time Step:  137
Prey:  2718
Intermediate-Predator:  533
Predator:  1607
Empty:  5142
-----------------------------------------
Time Step:  138
Prey:  2734
Intermediate-Predator:  542
Predator:  1614
Empty:  5110
-----------------------------------------
Time Step:  139
Prey:  2709
Intermediate-Predator:  554
Predator:  1615
Empty:  5122
-----------------------------------------
Time Step:  140
Prey:  2683
Intermediate-Predator:  573
Predator:  1610
Empty:  5134
-----------------------------------------
Time Step:  141
Prey:  2692
Intermediate-Predator:  593
Predator:  1575
Empty:  5140
--------------------------

Time Step:  200
Prey:  2532
Intermediate-Predator:  640
Predator:  1793
Empty:  5035
-----------------------------------------
Time Step:  201
Prey:  2483
Intermediate-Predator:  660
Predator:  1806
Empty:  5051
-----------------------------------------
Time Step:  202
Prey:  2464
Intermediate-Predator:  658
Predator:  1816
Empty:  5062
-----------------------------------------
Time Step:  203
Prey:  2484
Intermediate-Predator:  642
Predator:  1791
Empty:  5083
-----------------------------------------
Time Step:  204
Prey:  2467
Intermediate-Predator:  625
Predator:  1761
Empty:  5147
-----------------------------------------
Time Step:  205
Prey:  2423
Intermediate-Predator:  641
Predator:  1762
Empty:  5174
-----------------------------------------
Time Step:  206
Prey:  2434
Intermediate-Predator:  635
Predator:  1737
Empty:  5194
-----------------------------------------
Time Step:  207
Prey:  2471
Intermediate-Predator:  625
Predator:  1678
Empty:  5226
--------------------------

Time Step:  267
Prey:  2592
Intermediate-Predator:  569
Predator:  1771
Empty:  5068
-----------------------------------------
Time Step:  268
Prey:  2581
Intermediate-Predator:  557
Predator:  1794
Empty:  5068
-----------------------------------------
Time Step:  269
Prey:  2576
Intermediate-Predator:  543
Predator:  1816
Empty:  5065
-----------------------------------------
Time Step:  270
Prey:  2637
Intermediate-Predator:  508
Predator:  1775
Empty:  5080
-----------------------------------------
Time Step:  271
Prey:  2673
Intermediate-Predator:  504
Predator:  1753
Empty:  5070
-----------------------------------------
Time Step:  272
Prey:  2709
Intermediate-Predator:  513
Predator:  1748
Empty:  5030
-----------------------------------------
Time Step:  273
Prey:  2742
Intermediate-Predator:  496
Predator:  1748
Empty:  5014
-----------------------------------------
Time Step:  274
Prey:  2793
Intermediate-Predator:  504
Predator:  1771
Empty:  4932
--------------------------

...

Time Step:  798
Prey:  2778
Intermediate-Predator:  440
Predator:  1931
Empty:  4851
-----------------------------------------
Time Step:  799
Prey:  2836
Intermediate-Predator:  429
Predator:  1909
Empty:  4826
-----------------------------------------
Time Step:  800
Prey:  2882
Intermediate-Predator:  434
Predator:  1922
Empty:  4762
-----------------------------------------
Time Step:  801
Prey:  2878
Intermediate-Predator:  438
Predator:  1911
Empty:  4773
-----------------------------------------
Time Step:  802
Prey:  2921
Intermediate-Predator:  440
Predator:  1921
Empty:  4718
-----------------------------------------
Time Step:  803
Prey:  2928
Intermediate-Predator:  443
Predator:  1974
Empty:  4655
-----------------------------------------
Time Step:  804
Prey:  2953
Intermediate-Predator:  456
Predator:  2005
Empty:  4586
-----------------------------------------
Time Step:  805
Prey:  2902
Intermediate-Predator:  476
Predator:  2030
Empty:  4592
--------------------------

Time Step:  864
Prey:  2383
Intermediate-Predator:  561
Predator:  1983
Empty:  5073
-----------------------------------------
Time Step:  865
Prey:  2381
Intermediate-Predator:  548
Predator:  1969
Empty:  5102
-----------------------------------------
Time Step:  866
Prey:  2382
Intermediate-Predator:  549
Predator:  1948
Empty:  5121
-----------------------------------------
Time Step:  867
Prey:  2374
Intermediate-Predator:  530
Predator:  1924
Empty:  5172
-----------------------------------------
Time Step:  868
Prey:  2372
Intermediate-Predator:  500
Predator:  1932
Empty:  5196
-----------------------------------------
Time Step:  869
Prey:  2361
Intermediate-Predator:  484
Predator:  1922
Empty:  5233
-----------------------------------------
Time Step:  870
Prey:  2349
Intermediate-Predator:  474
Predator:  1928
Empty:  5249
-----------------------------------------
Time Step:  871
Prey:  2373
Intermediate-Predator:  472
Predator:  1920
Empty:  5235
--------------------------

Time Step:  931
Prey:  2515
Intermediate-Predator:  562
Predator:  1875
Empty:  5048
-----------------------------------------
Time Step:  932
Prey:  2510
Intermediate-Predator:  565
Predator:  1836
Empty:  5089
-----------------------------------------
Time Step:  933
Prey:  2514
Intermediate-Predator:  567
Predator:  1852
Empty:  5067
-----------------------------------------
Time Step:  934
Prey:  2481
Intermediate-Predator:  565
Predator:  1874
Empty:  5080
-----------------------------------------
Time Step:  935
Prey:  2489
Intermediate-Predator:  583
Predator:  1878
Empty:  5050
-----------------------------------------
Time Step:  936
Prey:  2494
Intermediate-Predator:  583
Predator:  1864
Empty:  5059
-----------------------------------------
Time Step:  937
Prey:  2510
Intermediate-Predator:  572
Predator:  1851
Empty:  5067
-----------------------------------------
Time Step:  938
Prey:  2509
Intermediate-Predator:  573
Predator:  1893
Empty:  5025
--------------------------

Time Step:  998
Prey:  2384
Intermediate-Predator:  531
Predator:  1820
Empty:  5265
-----------------------------------------
Time Step:  999
Prey:  2373
Intermediate-Predator:  521
Predator:  1841
Empty:  5265
-----------------------------------------
Time Step:  1000
Prey:  2378
Intermediate-Predator:  522
Predator:  1840
Empty:  5260
-----------------------------------------
Kill Count:  0
Prey Kill Count:  211067


KeyboardInterrupt: Interrupted by user